### Install sagemaker-experiments

In [1]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

In [2]:
import sagemaker
import boto3

session = sagemaker.Session()
sm = boto3.Session().client('sagemaker')
role = sagemaker.get_execution_role()

In [16]:
BUCKET_NAME = 'snowflake-getting-started'
BASE_PREFIX = 'bank-marketing'

TRAIN_DATA_PATH = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/train/train_data.csv'
TEST_DATA_PATH = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/test/test_data.csv'

EXPERIMENTS_OUTPUT_LOC = 's3://'+BUCKET_NAME+'/'+BASE_PREFIX+'/experiments-xboost'

print ('Experiment metadata would be published at -',EXPERIMENTS_OUTPUT_LOC)

Experiment metadata would be published at - s3://snowflake-getting-started/bank-marketing/experiments-xboost


### Step 1 - Setup an Experiment

### Create an Experiment

In [24]:
from smexperiments.experiment import Experiment
import time

EXPERIMENT_NAME=f"experiment-bank-marketing-xgboost-classification-model-{int(time.time())}"

iris_experiment = Experiment.create(
    experiment_name=EXPERIMENT_NAME,
    description="Bank marketing classification", 
    sagemaker_boto_client=sm)

EXP_DEBUGGING_OUTPUTS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/debugging'
EXP_TRAINED_MODELS=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/trained_models'
EXP_SOURCE_CODE=EXPERIMENTS_OUTPUT_LOC+'/'+EXPERIMENT_NAME+'/code'

print ('Experiment debugging data available at -',EXP_DEBUGGING_OUTPUTS)
print ('Experiment trained moddels available at -',EXP_TRAINED_MODELS)
print ('Experiment source code available at -', EXP_SOURCE_CODE)

Experiment debugging data available at - s3://snowflake-getting-started/bank-marketing/experiments-xboost/experiment-bank-marketing-xgboost-classification-model-1596131323/debugging
Experiment trained moddels available at - s3://snowflake-getting-started/bank-marketing/experiments-xboost/experiment-bank-marketing-xgboost-classification-model-1596131323/trained_models
Experiment source code available at - s3://snowflake-getting-started/bank-marketing/experiments-xboost/experiment-bank-marketing-xgboost-classification-model-1596131323/code


In [28]:
from sagemaker.xgboost import XGBoost
from smexperiments.trial import Trial
from sagemaker.debugger import rule_configs, Rule, DebuggerHookConfig, CollectionConfig

save_interval ='1'

for i, num_round in enumerate([5]):
    
    # create trial
    trial_name = f"trial-iris-classification-model-{num_round}-max-iter-{int(time.time())}"
    trial_metadata_prefix = BASE_PREFIX+'/'+EXPERIMENT_NAME+'/metadata/'+trial_name
    print ('Trial metadata would be stored at -',trial_metadata_prefix)
    
    bankmktg_trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=iris_experiment.experiment_name,
        sagemaker_boto_client=sm,
    )
    
    executor = XGBoost(entry_point='./train.py',
                      framework_version='1.0-1',
                      train_instance_type='ml.m4.xlarge',
                      train_instance_count=1,
                      sagemaker_session = session,
                      role = role,
                      code_location  = EXP_SOURCE_CODE,
                      hyperparameters = {
                          'num_round':num_round,
                          "objective":"binary:logistic"
                      },
                      input_mode='File',
                      enable_sagemaker_metrics=True,
                      debugger_hook_config=DebuggerHookConfig(
                                  s3_output_path=EXP_DEBUGGING_OUTPUTS, 
                                  hook_parameters={
                                    'save_interval': '1'
                                  },
                                  # Required - See https://github.com/awslabs/sagemaker-debugger/blob/master/docs/api.md#built-in-collections for supported collections
                                  collection_configs=[ 
                                      CollectionConfig( name="metrics"), 
                                      CollectionConfig( name="feature_importance"), 
                                      CollectionConfig( name="full_shap"), 
                                      CollectionConfig( name="average_shap"), 
                                  ],
                                ),
                      rules=[ 
                          Rule.sagemaker( 
                              rule_configs.loss_not_decreasing(), 
                              rule_parameters={ "collection_names": "metrics", "num_steps": str(save_interval * 2), }, 
                          ), 
                      ],
                      output_path = EXP_TRAINED_MODELS
                )
    
    trial_component_training_job = "training-job-bank-mktg-classification-model-{}".format(int(time.time()))
    executor.fit(
        inputs={'train': TRAIN_DATA_PATH, 'test': TEST_DATA_PATH},
        job_name=trial_component_training_job,
        experiment_config={
            "TrialName": bankmktg_trial.trial_name,
            "TrialComponentDisplayName": "Model-Training",
        },
        logs=True,
        #wait=True
    )
    
    # give it a while before dispatching the next training job
    time.sleep(2)

INFO:sagemaker:Creating training-job with name: training-job-bank-mktg-classification-model-1596132296


Trial metadata would be stored at - bank-marketing/experiment-bank-marketing-xgboost-classification-model-1596131784/metadata/trial-iris-classification-model-5-max-iter-1596132296
2020-07-30 18:04:56 Starting - Starting the training job...
2020-07-30 18:05:20 Starting - Launching requested ML instances
********* Debugger Rule Status *********
*
*  LossNotDecreasing: InProgress        
*
****************************************
......
2020-07-30 18:06:21 Starting - Preparing the instances for training......
2020-07-30 18:07:22 Downloading - Downloading input data
2020-07-30 18:07:22 Training - Downloading the training image...
2020-07-30 18:07:51 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module train does not pr

UnexpectedStatusException: Error for Training job training-job-bank-mktg-classification-model-1596132296: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/miniconda3/bin/python -m train --num_round 5 --objective binary:logistic"

In [ ]:
executor.latest_training_job.rule_job_summary()